In [1]:
# Math Operations
import numpy as np
from math import pi

# Data Preprocessing
import pandas as pd
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
!pip install soccerplots
import soccerplots
from soccerplots.radar_chart import Radar   # for custom radar visuals

# Machine Learning
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Print Statement
print('All necessary libraries imported.')

All necessary libraries imported.


In [2]:
leeds_s1 = pd.read_csv("datasets\cleanedbasic\leeds20-21.csv")
leeds_s2 = pd.read_csv("datasets\cleanedbasic\leeds21-22.csv")
# hudds
hudds_s1 = pd.read_csv("datasets\cleanedbasic\hudds17-18.csv")
hudds_s2 = pd.read_csv("datasets\cleanedbasic\hudds18-19.csv")
# brighton
bri_s1 = pd.read_csv("datasets/cleanedbasic/brighton17-18.csv")
bri_s2 = pd.read_csv("datasets/cleanedbasic/brighton18-19.csv")
# newcastle
new_s1 = pd.read_csv("datasets/cleanedbasic/newcastle17-18.csv")
new_s2 = pd.read_csv("datasets/cleanedbasic/newcastle18-19.csv")
# sheff utd
sheff_s1 = pd.read_csv("datasets/cleanedbasic/sheffutd19-20.csv")
sheff_s2 = pd.read_csv("datasets/cleanedbasic/sheffutd20-21.csv")
# aston villa
villa_s1 = pd.read_csv("datasets/cleanedbasic/villa19-20.csv")
villa_s2 = pd.read_csv("datasets/cleanedbasic/villa20-21.csv")
# wolves
wolves_s1 = pd.read_csv("datasets/cleanedbasic/wolves18-19.csv")
wolves_s2 = pd.read_csv("datasets/cleanedbasic/wolves19-20.csv")

In [3]:
season_1 = [leeds_s1, hudds_s1, bri_s1, new_s1, sheff_s1, villa_s1, wolves_s1]
season_2 = [leeds_s2, hudds_s2, bri_s2, new_s2, sheff_s2, villa_s2, wolves_s2]
teams = ['Leeds United', 'Huddersfield Town', 'Brighton', 'Newcastle United', 'Sheffield United', 'Aston Villa',
        'Wolves']

In [4]:
new_s1.tail()

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Per90G+A-PK,xG,npxG,xA,npxG+xAG,Per90xG,Per90xA,Per90xG+xA,Per90npxG,Per90npxG+xAG
24,Henri Saivet,sn SEN,MF,26.0,1,1,83,0.9,1,0,...,1.08,0.1,0.1,0.0,0.1,0.05,0.00,0.05,0.05,0.05
25,Aleksandar Mitrovi?,rs SRB,FW,22.0,6,0,95,1.1,1,0,...,0.95,1.2,1.2,0.2,1.3,1.14,0.15,1.29,1.14,1.29
26,Massadio Haïdara,ml MLI,FW,24.0,1,0,4,0.0,0,0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
27,Squad Total,NaN,NaN,25.6,38,418,3420,38.0,38,29,...,1.76,41.6,40.9,32.1,72.9,1.10,0.84,1.94,1.08,1.92
28,Opponent Total,NaN,NaN,26.9,38,418,3420,38.0,44,32,...,1.89,52.4,47.6,37.4,85.1,1.38,0.99,2.37,1.25,2.24


In [5]:
def replace_value(dfs):
    for i in range(len(dfs)):
            dfs[i].replace({'Player': {'Squad Total': teams[i]}}, inplace = True)

In [6]:
replace_value(season_1)
replace_value(season_2)

In [7]:
new_s1.loc[new_s1['Player'] == 'Newcastle United']

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Per90G+A-PK,xG,npxG,xA,npxG+xAG,Per90xG,Per90xA,Per90xG+xA,Per90npxG,Per90npxG+xAG
27,Newcastle United,NaN,NaN,25.6,38,418,3420,38.0,38,29,...,1.76,41.6,40.9,32.1,72.9,1.1,0.84,1.94,1.08,1.92


In [8]:
for i in range(len(season_1)):
    lst = []
    for j in range(len(season_1[i])):
        lst.append('Season 1')
    #print(len(lst))
    season_1[i]['Season'] = lst

In [9]:
for i in range(len(season_2)):
    lst = []
    for j in range(len(season_2[i])):
        lst.append('Season 2')
    #print(len(lst))
    season_2[i]['Season'] = lst

In [10]:
wolves_s1.columns

Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Per90Gls', 'Per90Ast',
       'Per90G+A', 'Per90G-PK', 'Per90G+A-PK', 'xG', 'npxG', 'xA', 'npxG+xAG',
       'Per90xG', 'Per90xA', 'Per90xG+xA', 'Per90npxG', 'Per90npxG+xAG',
       'Season'],
      dtype='object')

In [11]:
def create_2p_radar(df1,df2,player_1_season,player_1_colour,team,
                    player_2_season,
                    player_2_colour,):
    
    ### Season 1 
    df_radar_player_1 = df1.loc[df1['Player'] == team]
    
    ### Season 2 
    df_radar_player_2 = df2.loc[df2['Player'] == team]
    
    ### Union two DataFrames together
    df_radar = pd.concat([df_radar_player_1, df_radar_player_2])
    
    ## Define list of columns for radar
    lst_cols_radar = ['Player',
                      'Season',
                      'Per90xG',
                      'Gls',
                      'Per90xA',
                      'Ast',
                      'npxG+xAG',
                      'Per90G-PK',
                      'Per90npxG'
                     ]
    
    ## Select only columns of interest for radar
    df_radar = df_radar[lst_cols_radar]

    ## Rename columns
    df_radar.columns=['Team',
                      'Season',
                      'Per 90 Expected Goals',
                      'Goals Scored',
                      'Per 90 Expected Assists',
                      'Assists',
                      'Non Penalty xG + xA',
                      'Per 90 Goals minus Penalties',
                      'Per 90 Non Penalty Expected Goals'
                     ] 
    
    ## Reset index
    df_radar = df_radar.reset_index(drop=True)
    #print(df_radar)

    ## Radar visualisation
    
    ### Get parameters
    params = list(df_radar.columns)
    params = params[2:]
    
    ### Add ranges to list of tuple pairs
    ranges = []
    a_values = []
    b_values = []
    
    ###
    for x in params:
        a = min(df_radar[params][x])
        a = a - (a*.25)

        b = max(df_radar[params][x])
        b = b + (b*.25)

        ranges.append((a, b))
        #print(ranges)

    ###
    for x in range(len(df_radar['Season'])):
        if df_radar['Season'][x] == player_1_season:
            a_values = df_radar.iloc[x].values.tolist()
        if df_radar['Season'][x] == player_2_season:
            b_values = df_radar.iloc[x].values.tolist()
    
    ###
    a_values = a_values[2:]
    b_values = b_values[2:]
    #print(a_values)
    #print(b_values)

    ###
    values = [a_values, b_values]
    #print(values)

    ###
    title = dict(title_name = team,
                 title_color = player_1_colour,
                 subtitle_name = player_1_season,
                 subtitle_color = player_1_colour,
                 title_name_2 = team,
                 title_color_2 = player_2_colour,
                 subtitle_name_2 = player_2_season,
                 subtitle_color_2 = player_2_colour,
                 title_fontsize = 18,
                 subtitle_fontsize=15
                )
    
    ### Define fonts and colours
    background='#f7f7f7'    #'#313233'
    mpl.rcParams.update(mpl.rcParamsDefault)
    
    endnote = '@https://github.com/kiansweeney11'
    
    ###
    radar = Radar()

    ### Create figure
    fig, ax = radar.plot_radar(ranges=ranges,
                               params=params,
                               values=values,
                               radar_color=[player_1_colour, player_2_colour],
                               alphas=[0.5, 0.5],
                               endnote=endnote,
                               title=title,
                               compare=True)
    
    ### Change background colour
    fig.set_facecolor(background)
    return fig

In [12]:
create_2p_radar(leeds_s1, leeds_s2, 'Season 1', 'royalblue', 'Leeds United', 'Season 2', 'goldenrod')
plt.savefig('images/leeds-attack-radar.png', bbox_inches='tight')

findfont: Font family ['Liberation Serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Liberation Serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Liberation Serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Liberation Serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Liberation Serif'] not found. Falling back to DejaVu Sans.


In [13]:
create_2p_radar(sheff_s1, sheff_s2, 'Season 1', 'black', 'Sheffield United', 'Season 2', 'indianred')
plt.savefig('images/sheffu-attack-radar.png', bbox_inches='tight')

In [14]:
create_2p_radar(villa_s1, villa_s2, 'Season 1', 'dodgerblue', 'Aston Villa', 'Season 2', 'palevioletred')
plt.savefig('images/villa-attack-radar.png', bbox_inches='tight')

In [15]:
create_2p_radar(wolves_s1, wolves_s2, 'Season 1', 'darkorange', 'Wolves', 'Season 2', 'black')
plt.savefig('images/wolves-attack-radar.png', bbox_inches='tight')

In [16]:
create_2p_radar(new_s1, new_s2, 'Season 1', 'khaki', 'Newcastle United', 'Season 2', 'black')
### some null values in 2017-18 data needs addressing -> crdY column
plt.savefig('images/newcastle-attack-radar.png', bbox_inches='tight')

In [17]:
create_2p_radar(hudds_s1, hudds_s2, 'Season 1', 'palegoldenrod', 'Huddersfield Town', 'Season 2', 'cornflowerblue')
plt.savefig('images/hudds-attack-radar.png', bbox_inches='tight')

In [18]:
create_2p_radar(bri_s1, bri_s2, 'Season 1', 'limegreen', 'Brighton', 'Season 2', 'mediumslateblue')
plt.savefig('images/brighton-attack-radar.png', bbox_inches='tight')

In [19]:
### clustering on players?
### see if patterns emerge based on positions ? all poor second seasons emerge together? 
### correlation matrices at team level to see do correlations exist at team level between final team ranking and performance metrics
## pca of performance metrics to understand most important values behind team
## look at oppenent values? could be used as values for a teams squad total with regards to what is conceded